In [1]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import  DataLoader,SubsetRandomSampler,random_split
from sklearn.model_selection import KFold
import pandas as pd

## DATA PREPARE

### IMPORT DATA

#### Datasets

 - Trainning dataset: https://cloud.ipb.pt/f/657d534db56645059905/?dl=1
 - Evaluate dataset: https://cloud.ipb.pt/f/27e4d3ac75d2405aa770/?dl=1

##### Dataset dispositions

#### Transformation functions and hyperparams

Aqui optaremos por duas transformações de imagens de treinamento, em modelos como AlexNet, VGG que trabalham com imagens de escala maior usaremos *227x227 pixels* para treinamento do AlexNet.

Outra abordagem será utilizar imagens de menor escala, para um rede neural menor de desenvolvimento próprio, baseado em outros notebooks e estudos relacionados, para esse modelo será utilizado amostras de imagens com a escala de *64x64 pixels*.

In [2]:
batch_size = 10
classes = (1,0)

train_transform = transforms.Compose([
    transforms.Resize((64,64)),
    # transforms.Resize((224,224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

In [14]:
train_dataset = datasets.ImageFolder('./train', transform=train_transform)

train_size = int(0.9 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset_split, prediction_dataset = random_split(train_dataset, [train_size, test_size])

# DataLoaders for train and validation
train_loader = DataLoader(train_dataset_split, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(train_dataset_split, batch_size=batch_size, shuffle=False)

# After evaluation, use the prediction dataset
prediction_loader = DataLoader(prediction_dataset, batch_size=batch_size, shuffle=False)


## Define Models Architecture

#### Our CNN architecture to 64x64px image input

In [7]:
class ConvolutionNeuralNetwork(nn.Module):
    def __init__(self,num_classes=7):
        super(ConvolutionNeuralNetwork, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(24)
        self.conv2 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(24)

        self.pool1 = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(in_channels=24, out_channels=48, kernel_size=5, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(48)
        self.conv4 = nn.Conv2d(in_channels=48, out_channels=48, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(48)

        self.pool2 = nn.MaxPool2d(2,2)
        self.drop1=nn.Dropout(p=0.2)

        self.conv5 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(96)
        self.conv6 = nn.Conv2d(in_channels=96, out_channels=96, kernel_size=4, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(96)

        self.pool3 = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(96*5*5, 192)
        self.drop2=nn.Dropout(p=0.2)

        self.fc2 = nn.Linear(192, 96)
        self.fc3 = nn.Linear(96, num_classes)


    def forward(self, input):
        output = F.relu(self.bn1(self.conv1(input)))      
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool1(output)  

        output = F.relu(self.bn3(self.conv3(output)))      
        output = F.relu(self.bn4(self.conv4(output)))     
        output = self.pool2(output)      
        output = self.drop1(output)  

        output = F.relu(self.bn5(self.conv5(output)))      
        output = F.relu(self.bn6(self.conv6(output)))  
        output = self.pool3(output)  

        output = output.reshape(output.size(0),-1)

        output = F.relu(self.fc1(output))
        output = self.drop2(output)
        output = F.relu(self.fc2(output))
        output = self.fc3(output)

        return output

#### Alex Net architecture to 277x277px image input

In [6]:
class AlexNet(nn.Module):
    def __init__(self,num_classes=7):
        super(AlexNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=10, stride=4, padding=1)
        self.bn1 = nn.BatchNorm2d(96)
        self.pool1= nn.MaxPool2d(3,2)

        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(3,2)

        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(384)

        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(384)

        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(3,2)

        self.fc1 = nn.Linear(9216, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)


    def forward(self, input):
        output = F.relu(self.bn1(self.conv1(input)))   
        output = self.pool1(output)     
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool2(output)    

        output = F.relu(self.bn3(self.conv3(output)))     
        output = F.relu(self.bn4(self.conv4(output)))   
        output = F.relu(self.bn5(self.conv5(output))) 
        output = self.pool3(output)   

        output = output.reshape(output.size(0),-1)

        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))
        output = self.fc3(output)

        return output

### Model Definition

In [8]:
model = ConvolutionNeuralNetwork()

### Define loss function and optimizer

In [9]:
# accelerator = Accelerator()
# device = accelerator.device
# Define your execution device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Runing on: "+ ("cuda" if torch.cuda.is_available() else "cpu"))

learning_rate = 0.05

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate,  weight_decay = 0.01)

Runing on: cpu


### Model save

In [10]:
def saveModel():
    torch.save(model.state_dict(), "apurated_model_mycnn.pth")

## Training and Validation functions

In [17]:
def train_epoch(model,device,dataloader):
    train_loss,train_correct=0.0,0
    model.train()
    for images, labels in dataloader:

        images,labels = images.to(device),labels.to(device)

        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output,labels)

        loss.backward()
        # accelerator.backward(loss)

        optimizer.step()
        train_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(output.data, 1)
        train_correct += (predictions == labels).sum().item()

    return train_loss,train_correct
  
def valid_epoch(model,device,dataloader):
    valid_loss, val_correct = 0.0, 0
    model.eval()
    with torch.no_grad():
        for images, labels in dataloader:

            images,labels = images.to(device),labels.to(device)
            output = model(images)
            loss=loss_fn(output,labels)
            
            valid_loss+=loss.item()*images.size(0)
            scores, predictions = torch.max(output.data,1)
            val_correct+=(predictions == labels).sum().item()

    return valid_loss,val_correct

In [18]:
def train(num_epochs):
    history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}
    best_accuracy = 0.0

    model.to(device)
    

    test_sampler = SubsetRandomSampler(val_idx)
    train_samples =  SubsetRandomSampler(train_idx)

    test_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=test_sampler)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_samples)

    for epoch in range(num_epochs):
        train_loss, train_correct=train_epoch(model,device,train_loader)
        test_loss, test_correct=valid_epoch(model,device,test_loader)

        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100

        test_loss = test_loss / len(test_loader.sampler)
        test_acc = test_correct / len(test_loader.sampler) * 100

        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                                num_epochs,
                                                                                                                train_loss,
                                                                                                                test_loss,
                                                                                                                train_acc,
                                                                                                                test_acc))
        if train_acc > best_accuracy:
            saveModel()
            best_accuracy = train_acc
            print("Best Accuracy:{} %".format(best_accuracy))

        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)   

    df_history = pd.DataFrame(data=history)
    df_history.to_csv("historic_mycnn.csv", encoding='utf-8', index=False)

## Training

In [19]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

KeyError: 'allocated_bytes.all.current'

In [21]:
train(5)
print('Finished Training')

Fold 1
Epoch:1/5 AVG Training Loss:0.027 AVG Test Loss:0.107 AVG Training Acc 99.00 % AVG Test Acc 98.04 %
Best Accuracy:99.0 %
Epoch:2/5 AVG Training Loss:0.018 AVG Test Loss:0.121 AVG Training Acc 99.00 % AVG Test Acc 98.04 %
Epoch:3/5 AVG Training Loss:0.006 AVG Test Loss:0.124 AVG Training Acc 100.00 % AVG Test Acc 98.04 %
Best Accuracy:100.0 %
Epoch:4/5 AVG Training Loss:0.005 AVG Test Loss:0.101 AVG Training Acc 100.00 % AVG Test Acc 98.04 %
Epoch:5/5 AVG Training Loss:0.014 AVG Test Loss:0.271 AVG Training Acc 99.50 % AVG Test Acc 94.12 %
Fold 2
Epoch:1/5 AVG Training Loss:0.078 AVG Test Loss:0.009 AVG Training Acc 98.01 % AVG Test Acc 100.00 %
Epoch:2/5 AVG Training Loss:0.020 AVG Test Loss:0.095 AVG Training Acc 99.00 % AVG Test Acc 96.00 %
Epoch:3/5 AVG Training Loss:0.043 AVG Test Loss:0.083 AVG Training Acc 98.01 % AVG Test Acc 96.00 %
Epoch:4/5 AVG Training Loss:0.017 AVG Test Loss:0.153 AVG Training Acc 99.00 % AVG Test Acc 96.00 %
Epoch:5/5 AVG Training Loss:0.045 AVG Te

# TEST

In [23]:
test_transform = transforms.Compose([
    # transforms.Resize((227,227)),
    transforms.Resize((64,64)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

test_dataset = datasets.ImageFolder('./eval', transform=test_transform)
test_loaded = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

FileNotFoundError: Couldn't find any class folder in ./eval.

In [24]:
def test_model(model):
    y_true=[]
    y_pred=[]

    valid_loss, val_correct = 0.0, 0
    model.eval()
    with torch.no_grad():
        for images, labels in test_loaded:
            y_true.extend(labels.numpy())

            output = model(images)
            loss=loss_fn(output,labels)
            valid_loss+=loss.item()*images.size(0)
            scores, predictions = torch.max(output.data,1)
            val_correct+=(predictions == labels).sum().item()

            y_pred.extend(predictions.cpu().numpy())

    return valid_loss,val_correct, y_pred, y_true

In [25]:
def testBatch(model):
    model.eval()
    # get batch of images from the test DataLoader  
    images, labels = next(iter(test_loaded))
   
    # show all images as one image grid
    img = torchvision.utils.make_grid(images)     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
   
    # Show the real labels on the screen 
    print('Real labels: ', ' '.join('%5s' % classes[labels[j]] 
                               for j in range(batch_size)))
  
    with torch.no_grad():
        output = model(images)
        scores, predictions = torch.max(output.data,1)
        
        # Let's show the predicted labels on the screen to compare with the real ones
        print('Predicted: ', ' '.join('%5s' % classes[predictions[j]] 
                                for j in range(batch_size)))

### Teste para arquitetura AlexNet

In [ ]:

model = AlexNet()
path = "apurated_model_alex.pth"
model.load_state_dict(torch.load(path))

In [ ]:
valid_loss,val_correct,y_pred,y_true = test_model(model)

valid_loss = valid_loss / len(test_loaded.sampler)
val_acc = val_correct / len(test_loaded.sampler) * 100

print("AVG Test Loss:{:.3f} AVG Test Acc {:.2f} %".format(valid_loss,val_acc))
                                    

#### Creating the confusion matrix

In [18]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cf_matrix = confusion_matrix(y_true, y_pred)

##### Mounting the confusion tabel to see error per class

A linha indica o valor real da classe e as colunas as predições, então por exemplo, na celula [2][3], quando a classe real era *happy* ele previu 0 imagens como *neutral*.

In [ ]:
dataframe = pd.DataFrame(cf_matrix, index=classes, columns=classes)
dataframe

In [ ]:
for classlabel in classes:
    percent_value =((dataframe[classlabel][classlabel])/100)*100
    print("Para classe {}, a accurácia foi de {:.2f} %".format(classlabel,percent_value))

In [ ]:
plt.figure(figsize=(8, 6))
 
# Create heatmap
sns.heatmap(dataframe, annot=True, cbar=None,cmap="YlGnBu",fmt="d")
 
plt.title("Confusion Matrix"), plt.tight_layout()
 
plt.ylabel("True Class"), 
plt.xlabel("Predicted Class")
plt.show()

### Teste para nossa arquitetura CNN

In [ ]:
model = ConvolutionNeuralNetwork()
path = "apurated_model_mycnn.pth"
model.load_state_dict(torch.load(path))

valid_loss,val_correct,y_pred,y_true = test_model(model)

valid_loss = valid_loss / len(test_loaded.sampler)
val_acc = val_correct / len(test_loaded.sampler) * 100

print("AVG Test Loss:{:.3f} AVG Test Acc {:.2f} %".format(valid_loss,val_acc))

#### Creating the confusion matrix

In [ ]:
cf_matrix = confusion_matrix(y_true, y_pred)
dataframe = pd.DataFrame(cf_matrix, index=classes, columns=classes)
dataframe

In [ ]:
for classlabel in classes:
    percent_value =((dataframe[classlabel][classlabel])/100)*100
    print("Para classe {}, a accurácia foi de {:.2f} %".format(classlabel,percent_value))

plt.figure(figsize=(8, 6))
 
# Create heatmap
sns.heatmap(dataframe, annot=True, cbar=None,cmap="YlGnBu",fmt="d")
 
plt.title("Confusion Matrix"), plt.tight_layout()
 
plt.ylabel("True Class"), 
plt.xlabel("Predicted Class")
plt.show()